In [1]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from urllib.request import urlopen as uReq
from tabula import read_pdf
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

### Problem no. 1
- Search for ONGC’s official website from a search engine.

In [2]:
searchstring = input() # Input from user: ONGC

ONGC


In [3]:
url = "https://www.google.com/search?q=" + searchstring
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url) 
page = driver.page_source # Getting page from url
driver.quit()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


### Problem 2
- From the search engine results

**2.1 Find link to official website**  

In [4]:
html = bs(page, "html.parser") # Parsing the webpage as HTML
imp_box = html.findAll("div",{"class":"BYM4Nd"}) # Fetching division tab in HTML file where official link resides.
official_link = imp_box[0].div.div.link['href'] # findAll returns a list and 1st element in that list contains href.
print("Official Link: {}".format(official_link))

Official Link: https://www.ongcindia.com/


**2.2 Use link to enter ONGC website to search for 2021 ONGC’s annual report**

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(official_link) # Opening Official Link
homepage = driver.page_source
homepage_html = bs(homepage, "html.parser") # parsing the homepage of ONGC as HTML
Annual_reports_section = homepage_html.findAll("div",{"class":"col-md-3 col-sm-6 post"})
Annual_report_link = official_link + Annual_reports_section[0].a['href']
driver.quit()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(Annual_report_link)
report_section = driver.page_source
report_section_html = bs(report_section, "html.parser")
report_section_box = report_section_html.findAll("div",{"class":"blog"})
report_section_link = official_link + report_section_box[0].p.map.area['href']
driver.quit()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(report_section_link)
report = driver.page_source
report_html = bs(report, "html.parser")
report_box = report_html.findAll("ul",{"class":"pressre1"})
n=2
report_link = official_link + report_box[0].select("li:nth-of-type("+str(n)+")")[0].a['href']
driver.quit()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


### Problem 3  
- Download ONGC’s 2021 annual report, in pdf format, to the “Download” folder of your local computer.

In [8]:
report = requests.get(report_link)
open(searchstring+'_AnnualReport'+'.pdf', 'wb').write(report.content)

27561447

### Problem no. 4
- Extract the information under the section where it shows all subsidiaries, joint ventures, and associates but only the following columns are relevant:  
1) Name of the Company  
2) Country of Incorporation  
3) Proportion of ownership interest as March 31, 2021  


In [9]:
df = read_pdf(r"C:\Users\Lenovo\ONGC_AnnualReport.pdf", pages = [451,452,453],lattice= True)
# Reading download pdf 
# Storing page nos. 451,452,453 in df

In [10]:
# Creating Dataframes for tables on each page
table1 = pd.DataFrame(df[1].iloc[2:]).reset_index(drop=True) 
table2 = pd.DataFrame(df[2].iloc[1:]).drop(index = [8]).reset_index(drop=True)
table3 = pd.DataFrame(df[4].iloc[1:]).reset_index(drop=True)

In [11]:
# Cleaning Data manually since some of the fields in table did not contain correct values.
table2.loc[19,'Proportion of ow'] = '13.00%'
table3.loc[12, 'Name of the Company'] = 'Pawan Hans Ltd. (PHL)'
table3.loc[12, 'Country of\rIncorporation'] = 'India'
table3.loc[12, 'Proportion of ow'] = '49.00%'

In [12]:
# Fetching required columns and renaming them
table1 = table1[['Name of the Company','Country of\rIncorporation','Proportion of ow']]
table1.columns = ['Name of the Company','Country','Interest']
table2 = table2[['Name of the Company','Country of\rIncorporation','Proportion of ow']]
table2.columns = ['Name of the Company','Country','Interest']
table3 = table3[['Name of the Company','Country of\rIncorporation','Proportion of ow']]
table3.columns = ['Name of the Company','Country','Interest']

In [13]:
# Concatenating these tables to form master_1
master_1 = pd.concat([table1,table2,table3], ignore_index = True, axis = 0)

# Replacing '\r' with space
for cols in list(master_1.columns):
    master_1[cols] = master_1[cols].str.replace('\r',' ')

In [14]:
master_1

,Name of the Company,Country,Interest
0,ONGC Videsh Limited (OVL),India,100%
1,ONGC Nile Ganga B.V.,The Netherlands,Class A : 100% Class B : 100% Class C : 55%
2,ONGC Campos Ltda.,Brazil,100%
3,ONGC Nile Ganga (San Cristobal) B.V.,The Netherlands,100%
4,ONGC Narmada Limited,Nigeria,100%
...,...,...,...
74,"Carabobo Ingenieria Y Construcciones, S.A (thr...",Venezuela,37.93%
75,Falcon Oil & Gas B.V. (through OVL),Netherlands,40.00%
76,Moz LNG I Holding Company Ltd (through OVL),Abu Dhabi,16.00%
77,GSPL India Gasnet Ltd.(through HPCL),India,11.00%


In [25]:
set(master_1['Name of the Company'])

{'Aavantika Gas Ltd. (through HPCL)',
 'Adani Petronet Dahej Port Pvt Ltd (APPPL) (through PLL)',
 'Beas Rovuma Energy Mozambique Ltd.',
 'Bhagyanagar Gas Ltd. (through HPCL) (Note No. 4(k))',
 'Biancus Holdings Limited',
 'Carabobo Ingenieria Y Construcciones, S.A (through OVL)',
 'Carabobo One AB',
 'Dahej SEZ Limited (DSEZ)',
 'Falcon Oil & Gas B.V. (through OVL)',
 'GSPL India Gasnet Ltd.(through HPCL)',
 'GSPL India Transco Ltd. (through HPCL)',
 'Godavari Gas Pvt Ltd. (through HPCL)',
 'HPCL Bio Fuels Ltd.',
 'HPCL Middle East FZCO (Note No. 4(h))',
 'HPCL Mittal Energy Ltd. (through HPCL) (Note No. 4(g))',
 'HPCL Rajasthan  refinery Ltd. (through HPCL)',
 'HPCL Shapoorji Energy Pvt Ltd. (HSEL) (Note No. 4(l))',
 'HPOIL Gas Pvt Ltd (through HPCL)',
 'Himalaya Energy Syria BV(through OVL)',
 'Hindustan Colas Pvt. Ltd. (through HPCL) (Note No. 4(i))',
 'Hindustan Petroleum Corporation Ltd (HPCL) (Note No. 4(c))',
 'IHB Pvt Ltd (through HPCL)',
 'Imperial Energy (Cyprus) Limited',
 

In [15]:
# Converting and storing master_1 in excel format in local system.
master_1.to_excel('master_1.xlsx', index = False)

### Problem no. 6

In [16]:
df1 = read_pdf(r"C:\Users\Lenovo\ongc_AnnualReport.pdf", pages = [374,375,376,377],lattice= True)
df2 = read_pdf(r"C:\Users\Lenovo\ongc_AnnualReport.pdf", pages = [379,380,381],lattice= True)
df3 = read_pdf(r"C:\Users\Lenovo\ongc_AnnualReport.pdf", pages = [382,383,384],lattice= True)

In [17]:
table1 = pd.DataFrame(df1[2].loc[1:]).reset_index(drop=True)
table1.columns = df1[2].iloc[0]
table2 = pd.DataFrame(df1[4].loc[1:]).reset_index(drop=True)[['Unnamed: 1','As at 31.03.2021AN']]
table3 = pd.DataFrame(df1[8].loc[1:]).reset_index(drop=True)[['Unnamed: 1','As at 31.03.2021']]
table4 = pd.DataFrame(df1[10].loc[1:]).reset_index(drop=True)[['Unnamed: 1','As at 31.03.2021']]

In [18]:
table1.rename(columns = {'Name of the\rsubsidiary':'Name of the Company','Share\rcapital':'Capital'}, inplace = True)
table1 = table1[['Name of the Company','Capital']]
table2.columns = ['Name of the Company','Capital']
table3.columns = ['Name of the Company','Capital']
table4.columns = ['Name of the Company','Capital']

In [19]:
master_2 = pd.concat([table1,table2,table3,table4], ignore_index = True, axis = 0)
master_2['Name of the Company'] = master_2['Name of the Company'].str.replace('\r',' ')
master_2

,Name of the Company,Capital
0,ONGC Videsh Limited,"150,000.00"
1,Mangalore Refinery & Petrochemicals Limited,"17,526.64"
2,Hindustan Petroleum Corporation Limited,"14,524.10"
3,ONGC Manga- lore Petrochem- icals Limited,"25,442.91"
4,ONGC Nile Ganga B.V.,5.30
5,ONGC Campos Ltda.,"30,989.87"
6,ONGC Nile Ganga (San Cristobal) B.V.,4.65
7,ONGC Amazon Alaknanda Limited,"9,187.21"
8,ONGC Narmada Limited,11.44
9,ONGC (BTC) Limited,71.56


### Problem 5
### Fuzzy Matching

In [21]:
master_2['Match'] = 0
master_2['Matched subsidiary name'] = 0

for i in range(len(master_2['Name of the Company'])):
    pattern = master_2['Name of the Company'].str.split(' ')[i][0] # Extracting 1st word from company name.
    match = list(master_2['Name of the Company'].str.find(pattern)) 
    indices = [i for i, x in enumerate(match) if x == 0]
    indices.remove(i) # Takes care of the case when 1st word in the name of company matches with itself.

    if indices: # If Indices list is not blank then perform operation else add Blank.
        matched_subsidiaries = list(master_2['Name of the Company'].iloc[indices])
        master_2['Match'].iloc[i] = "Yes"
        master_2['Matched subsidiary name'].iloc[i] = matched_subsidiaries    
    else:
        master_2['Match'].iloc[i] = "No" 
        master_2['Matched subsidiary name'].iloc[i] = '-'
        

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
master_2

,Name of the Company,Capital,Match,Matched subsidiary name
0,ONGC Videsh Limited,"150,000.00",Yes,"[ONGC Manga- lore Petrochem- icals Limited, ON..."
1,Mangalore Refinery & Petrochemicals Limited,"17,526.64",No,-
2,Hindustan Petroleum Corporation Limited,"14,524.10",No,-
3,ONGC Manga- lore Petrochem- icals Limited,"25,442.91",Yes,"[ONGC Videsh Limited, ONGC Nile Ganga B.V., ON..."
4,ONGC Nile Ganga B.V.,5.30,Yes,"[ONGC Videsh Limited, ONGC Manga- lore Petroch..."
5,ONGC Campos Ltda.,"30,989.87",Yes,"[ONGC Videsh Limited, ONGC Manga- lore Petroch..."
6,ONGC Nile Ganga (San Cristobal) B.V.,4.65,Yes,"[ONGC Videsh Limited, ONGC Manga- lore Petroch..."
7,ONGC Amazon Alaknanda Limited,"9,187.21",Yes,"[ONGC Videsh Limited, ONGC Manga- lore Petroch..."
8,ONGC Narmada Limited,11.44,Yes,"[ONGC Videsh Limited, ONGC Manga- lore Petroch..."
9,ONGC (BTC) Limited,71.56,Yes,"[ONGC Videsh Limited, ONGC Manga- lore Petroch..."


In [23]:
master_2.to_excel('master_2.xlsx', index = False)